<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load-the-libraries" data-toc-modified-id="Load-the-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="#missing-values-imputation" data-toc-modified-id="missing-values-imputation-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>missing values imputation</a></span></li><li><span><a href="#Data-Types" data-toc-modified-id="Data-Types-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Data Types</a></span></li></ul></li><li><span><a href="#Modelling" data-toc-modified-id="Modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><span><a href="#Train-validation-split" data-toc-modified-id="Train-validation-split-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Train validation split</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Xgboost" data-toc-modified-id="Xgboost-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Xgboost</a></span></li><li><span><a href="#Hyperparameter-tuning" data-toc-modified-id="Hyperparameter-tuning-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Hyperparameter tuning</a></span></li></ul></li></ul></div>

# Description
Reference: https://datahack.analyticsvidhya.com/contest/all/  


**Predict Loan Eligibility for Dream Housing Finance company**
Dream Housing Finance company deals in all kinds of home loans. They have presence across all urban, semi urban and rural areas. Customer first applies for home loan and after that company validates the customer eligibility for loan.

Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have provided a dataset to identify the customers segments that are eligible for loan amount so that they can specifically target these customers. 

**Data Dictionary**
Train file: CSV containing the customers for whom loan eligibility is known as 'Loan_Status'

| Variable | Description |
| :---|:---|
| Loan_ID | Unique Loan ID |
| Gender | Male/ Female |
| Married | Applicant married (Y/N) |
| Dependents | Number of dependents |
| Education | Applicant Education (Graduate/ Under Graduate) |
| Self_Employed | Self employed (Y/N) |
| ApplicantIncome | Applicant income |
| CoapplicantIncome | Coapplicant income |
| LoanAmount | Loan amount in thousands |
| Loan_Amount_Term | Term of loan in months |
| Credit_History | credit history meets guidelines |
| Property_Area | Urban/ Semi Urban/ Rural |
| Loan_Status | (Target) Loan approved (Y/N) |


**Evaluation Metric**  
Your model performance will be evaluated on the basis of your prediction of loan status for the test data (test.csv), which contains similar data-points as train except for the loan status to be predicted. Your submission needs to be in the format as shown in sample submission.

We at our end, have the actual loan status for the test dataset, against which your predictions will be evaluated. We will use the Accuracy value to judge your response.



**Public and Private Split**   
Test file is further divided into Public (25%) and Private (75%)

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

# Load the libraries

In [55]:
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.plotting.backend = "matplotlib"
pd.set_option('max_columns',100)

import time,os,json
time_start_notebook = time.time()
home = os.path.expanduser('~')
SEED=100

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

[(x.__name__,x.__version__) for x in [np,pd,sns]]

[('numpy', '1.18.4'), ('pandas', '1.1.0'), ('seaborn', '0.10.1')]

In [106]:
import sklearn
import xgboost as xgb
import lightgbm as lgb
import catboost as cb


# Load the data

In [57]:
df_train = pd.read_csv('../data/raw/train.csv')
print(df_train.shape)
df_train.head()

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [58]:
df_test = pd.read_csv('../data/raw/test.csv')
print(df_test.shape)
df_test.head()

(367, 12)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


# Data Processing

In [59]:
df = df_train.copy()
df = df.drop('Loan_ID',axis=1)

In [60]:
df.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [61]:
df.apply(pd.Series.unique)

Gender                                             [Male, Female, nan]
Married                                                 [No, Yes, nan]
Dependents                                          [0, 1, 2, 3+, nan]
Education                                     [Graduate, Not Graduate]
Self_Employed                                           [No, Yes, nan]
ApplicantIncome      [5849, 4583, 3000, 2583, 6000, 5417, 2333, 303...
CoapplicantIncome    [0.0, 1508.0, 2358.0, 4196.0, 1516.0, 2504.0, ...
LoanAmount           [nan, 128.0, 66.0, 120.0, 141.0, 267.0, 95.0, ...
Loan_Amount_Term     [360.0, 120.0, 240.0, nan, 180.0, 60.0, 300.0,...
Credit_History                                         [1.0, 0.0, nan]
Property_Area                                [Urban, Rural, Semiurban]
Loan_Status                                                     [Y, N]
dtype: object

## missing values imputation

In [62]:
c = 'Gender'
df[c].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [63]:
# fill with mode
df[c] = df[c].fillna(df[c].mode()[0])
df[c].value_counts()

Male      502
Female    112
Name: Gender, dtype: int64

In [64]:
c = 'Married'
df[c].value_counts()

Yes    398
No     213
Name: Married, dtype: int64

In [65]:
pd.crosstab(df['Married'],df['Education'],margins=True)

Education,Graduate,Not Graduate,All
Married,,,
No,168,45,213
Yes,309,89,398
All,477,134,611


In [66]:
# choose graduate ==> married

cond = (df['Education']=='Graduate') & (df['Married'].isnull()) 
df.loc[cond, 'Married'] = 'Yes'

cond = (df['Education']!='Graduate') & (df['Married'].isnull()) 
df.loc[cond, 'Married'] = 'No'

In [67]:
df['Married'].unique()

array(['No', 'Yes'], dtype=object)

In [68]:
df['Dependents'].value_counts(dropna=False)

0      345
1      102
2      101
3+      51
NaN     15
Name: Dependents, dtype: int64

In [69]:
c = 'Dependents'
df[c] = df[c].fillna(df[c].mode()[0])
df[c].value_counts(dropna=False)

0     360
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [70]:
df['Self_Employed'].value_counts(dropna=False)

No     500
Yes     82
NaN     32
Name: Self_Employed, dtype: int64

In [71]:
c = 'Self_Employed'
df[c] = df[c ].fillna(df[c].mode()[0])
df[c].value_counts(dropna=False)

No     532
Yes     82
Name: Self_Employed, dtype: int64

In [72]:
df['Credit_History'].value_counts(dropna=False)

1.0    475
0.0     89
NaN     50
Name: Credit_History, dtype: int64

In [73]:
c = 'Credit_History'
df[c] = df[c ].fillna(df[c].mode()[0])
df[c].value_counts(dropna=False)

1.0    525
0.0     89
Name: Credit_History, dtype: int64

In [74]:
c = 'LoanAmount'
df[c] = df[c].fillna(df[c].mean())
df[c].isna().sum()

0

In [75]:
c = 'Loan_Amount_Term'
df[c] = df[c].fillna(df[c].mean())
df[c].isna().sum()

0

In [76]:
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

## Data Types

In [77]:
df.apply(pd.Series.unique)

Gender                                                  [Male, Female]
Married                                                      [No, Yes]
Dependents                                               [0, 1, 2, 3+]
Education                                     [Graduate, Not Graduate]
Self_Employed                                                [No, Yes]
ApplicantIncome      [5849, 4583, 3000, 2583, 6000, 5417, 2333, 303...
CoapplicantIncome    [0.0, 1508.0, 2358.0, 4196.0, 1516.0, 2504.0, ...
LoanAmount           [146.41216216216216, 128.0, 66.0, 120.0, 141.0...
Loan_Amount_Term     [360.0, 120.0, 240.0, 342.0, 180.0, 60.0, 300....
Credit_History                                              [1.0, 0.0]
Property_Area                                [Urban, Rural, Semiurban]
Loan_Status                                                     [Y, N]
dtype: object

In [78]:
df.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [79]:
df['Gender'] = df['Gender'].map({'Male':1, 'Female': 0})
df['Married'] = df['Married'].map({'Yes':1, 'No': 0 })
df['Education'] = df['Education'].map({'Graduate': 1, 'Not Graduate': 0})
df['Self_Employed'] = df['Self_Employed'].map({'Yes':1, 'No': 0})
df['Loan_Status'] = df['Loan_Status'].map({'Y':1, 'N': 0})

df.apply(pd.Series.unique)

Gender                                                          [1, 0]
Married                                                         [0, 1]
Dependents                                               [0, 1, 2, 3+]
Education                                                       [1, 0]
Self_Employed                                                   [0, 1]
ApplicantIncome      [5849, 4583, 3000, 2583, 6000, 5417, 2333, 303...
CoapplicantIncome    [0.0, 1508.0, 2358.0, 4196.0, 1516.0, 2504.0, ...
LoanAmount           [146.41216216216216, 128.0, 66.0, 120.0, 141.0...
Loan_Amount_Term     [360.0, 120.0, 240.0, 342.0, 180.0, 60.0, 300....
Credit_History                                              [1.0, 0.0]
Property_Area                                [Urban, Rural, Semiurban]
Loan_Status                                                     [1, 0]
dtype: object

In [80]:
# one hot encoding
cols = ['Dependents','Property_Area']
df = pd.get_dummies(df,columns=cols,drop_first=True)
df.head()

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Semiurban,Property_Area_Urban
0,1,0,1,0,5849,0.0,146.412162,360.0,1.0,1,0,0,0,0,1
1,1,1,1,0,4583,1508.0,128.000000,360.0,1.0,0,1,0,0,0,0
2,1,1,1,1,3000,0.0,66.000000,360.0,1.0,1,0,0,0,0,1
3,1,1,0,0,2583,2358.0,120.000000,360.0,1.0,1,0,0,0,0,1
4,1,0,1,0,6000,0.0,141.000000,360.0,1.0,1,0,0,0,0,1


In [81]:
df.sum().sum()

4616154.987556207

In [82]:
def clean_data(df):
    df = df.copy()
    # drop unwanted features
    df = df.drop('Loan_ID',axis=1)

    # missing values imputation
    ## fill married yes no from graudated or not
    cond = (df['Education']=='Graduate') & (df['Married'].isnull()) 
    df.loc[cond, 'Married'] = 'Yes'
    cond = (df['Education']!='Graduate') & (df['Married'].isnull()) 
    df.loc[cond, 'Married'] = 'No'

    ## fill with mode
    cols_mode = ['Gender', 'Dependents', 'Self_Employed', 'Credit_History']
    for c in cols_mode:
        df[c] = df[c].fillna(df[c].mode()[0])

    ## fill with mean
    cols_mean = ['LoanAmount','Loan_Amount_Term' ]
    for c in cols_mean:
        df[c] = df[c].fillna(df[c].mean())

    # mapping string to integers
    df['Gender'] = df['Gender'].map({'Male':1, 'Female': 0})
    df['Married'] = df['Married'].map({'Yes':1, 'No': 0 })
    df['Education'] = df['Education'].map({'Graduate': 1, 'Not Graduate': 0})
    df['Self_Employed'] = df['Self_Employed'].map({'Yes':1, 'No': 0})
    
    # target 
    target = 'Loan_Status'
    if target in df.columns:
        df[target] = df[target].map({'Y':1, 'N': 0})

    # one hot encoding
    cols = ['Dependents','Property_Area']
    df = pd.get_dummies(df,columns=cols,drop_first=True)
    return df

In [83]:
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')

df_train = clean_data(df_train)
df_test = clean_data(df_test)

print(df_train.shape)
print(df_test.shape)
df_train.head(2).append(df_train.tail(2))

(614, 15)
(367, 14)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Semiurban,Property_Area_Urban
0,1,0,1,0,5849,0.0,146.412162,360.0,1.0,1,0,0,0,0,1
1,1,1,1,0,4583,1508.0,128.000000,360.0,1.0,0,1,0,0,0,0
612,1,1,1,0,7583,0.0,187.000000,360.0,1.0,1,0,1,0,0,1
613,0,0,1,1,4583,0.0,133.000000,360.0,0.0,0,0,0,0,1,0


# Modelling

## Train validation split

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
df_train_orig = df_train.copy()
target = 'Loan_Status'
df_Xtrain, df_Xvalid, ser_ytrain, ser_yvalid = train_test_split(
    df_train_orig.drop(target,axis=1), df_train_orig[target],
    test_size = 0.2,
    random_state=SEED,
    stratify=df[target]
)

ytrain = ser_ytrain.to_numpy().ravel()
yvalid = ser_yvalid.to_numpy().ravel()

print(f'train shape: {df_Xtrain.shape}')
print(f'valid shape: {df_Xvalid.shape}')

df_Xtrain.head(2)

train shape: (491, 14)
valid shape: (123, 14)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Semiurban,Property_Area_Urban
203,1,1,0,0,3500,1083.0,135.0,360.0,1.0,1,0,0,0,1
369,1,1,1,0,19730,5266.0,570.0,360.0,1.0,0,0,0,0,0


In [86]:
Xtr = df_Xtrain
ytr = ytrain
Xvd = df_Xvalid
yvd = yvalid

In [87]:
from sklearn import metrics

df_eval = pd.DataFrame({
    'Model': [],
    'Description': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F-score': []
})

## Random Forest

In [88]:
from sklearn.ensemble import RandomForestClassifier

In [89]:
model = RandomForestClassifier(n_estimators=500, n_jobs=-1,random_state=SEED)
model

RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=100)

In [90]:
model = RandomForestClassifier(n_estimators=500, n_jobs=-1,random_state=SEED)
model.fit(Xtr,ytr)

vd_preds = model.predict(Xvd)

acc = metrics.accuracy_score(yvd, vd_preds)
pre = metrics.precision_score(yvd, vd_preds)
rec = metrics.recall_score(yvd, vd_preds)
f1  = metrics.f1_score(yvd,vd_preds)

row = ['RF', 'default', acc, pre, rec, f1]
df_eval.loc[len(df_eval)] = row
df_eval = df_eval.drop_duplicates(['Model','Description'])
df_eval

,Model,Description,Accuracy,Precision,Recall,F-score
0,RF,default,0.804878,0.796117,0.964706,0.87234


## Xgboost

In [91]:
from xgboost import XGBClassifier

In [92]:
model = XGBClassifier(random_state=SEED,n_jobs=-1)

model.fit(Xtr,ytr)
vd_preds = model.predict(Xvd)

acc = metrics.accuracy_score(yvd, vd_preds)
pre = metrics.precision_score(yvd, vd_preds)
rec = metrics.recall_score(yvd, vd_preds)
f1  = metrics.f1_score(yvd,vd_preds)

row = ['XGB', 'default', acc, pre, rec, f1]
df_eval.loc[len(df_eval)] = row
df_eval = df_eval.drop_duplicates(['Model','Description'])
df_eval

,Model,Description,Accuracy,Precision,Recall,F-score
0,RF,default,0.804878,0.796117,0.964706,0.872340
1,XGB,default,0.829268,0.807692,0.988235,0.888889


## Hyperparameter tuning

```
XGBClassifier?
XGBClassifier(
    max_depth=3,
    learning_rate=0.1,
    n_estimators=100,
    verbosity=1,
    silent=None,
    objective='binary:logistic',
    booster='gbtree',
    n_jobs=1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=1,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,
    missing=None,
```

In [115]:
params = dict(
max_depths = [3,4,5,6,7,8,9,10],
learning_rate = [0.001, 0.01, 0.1, 0.2],
n_estimators = [50,100, 120, 150, 200,1000],
gamma = [0, 0.01, 0.1],
subsample = [0.6, 0.7,0.8,0.9,1],
colsample_by_tree = [0.6, 0.7,0.8,0.9,1],
reg_alpha = [0.001, 0.01, 0.1, 1],
reg_lambda = [0.001, 0.01, 0.1, 1],
)

params

{'max_depths': [3, 4, 5, 6, 7, 8, 9, 10],
 'learning_rate': [0.001, 0.01, 0.1, 0.2],
 'n_estimators': [50, 100, 120, 150, 200, 1000],
 'gamma': [0, 0.01, 0.1],
 'subsample': [0.6, 0.7, 0.8, 0.9, 1],
 'colsample_by_tree': [0.6, 0.7, 0.8, 0.9, 1],
 'reg_alpha': [0.001, 0.01, 0.1, 1],
 'reg_lambda': [0.001, 0.01, 0.1, 1]}

In [111]:
from sklearn.model_selection import (GridSearchCV,
                                     RandomizedSearchCV,
                                     StratifiedKFold)

In [ ]:
%%time
model = XGBClassifier(random_state=SEED,n_jobs=-1)
grid = GridSearchCV(model,params,cv=5,scoring='accuracy',verbose=0)
grid.fit(Xtr,ytr)